### Import Statements

In [1]:
#!conda install -c conda-forge geocoder -y
#!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py36_0   conda-forge
    ratelim:    0.1.6-py36_0 conda-forge


orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB     | ##################################### | 100% 
ratelim-0.1.6      

In [78]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print("soup's ready!")

soup's ready!


### Get data and clean it up

In [3]:
#Scrape Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = requests.get(url)
data = request.text

#Clean html with beautifulsoup and find the table
soup = BeautifulSoup(data, 'html.parser')
table = soup.table
table_rows = table.find_all('tr')

#retrieve table rows and append them to a list
table_list = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    table_list.append(row)

#remove empty first row
del table_list[0]

#remove line breaks between rows
for index, entry in enumerate(table_list):
    table_list[index][2] = table_list[index][2].rstrip('\n')

### Create DataFrame

In [4]:
df = pd.DataFrame(table_list, columns= ['PostalCode', 'Borough', 'Neighborhood'])
#remove rows where borough is undefined
df = df[df.Borough != 'Not assigned']

#assigen borough for neighborhoods where value is 'not assigned' 
for index, row in df.iterrows():
    if row[2] == "Not assigned":
        row[2] = row[1]

#group neighborhoods into the same postal code
df = df.groupby('PostalCode').agg({'Borough':'first', 'Neighborhood':', '.join})
df = df.reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Print DataFrame Shape

In [5]:
df.shape

(103, 3)

### Get Latitude and Longitude Data

Geocoder would not work for me. It was taking a rediculous amount of time to load and used all my free space in watson studios.

Due to this, I opted to use the csv.

In [7]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Got it!')

Got it!


### Lat/Long DataFrame

In [8]:
#create dataframe of lat and long
df_lat_lon = pd.read_csv('Geospatial_Coordinates.csv')
#merge with previous dataframe
df_lat_lon.rename(columns= {'Postal Code': 'PostalCode'}, inplace = True)
df_whole = pd.merge(df, df_lat_lon, on = 'PostalCode')
df_whole.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Toronto Clustering

Map of all Toronto boroguhs to first visulaize all the data.

In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_whole['Latitude'], df_whole['Longitude'], df_whole['Borough'], df_whole['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For this project, I will be only looking a the borough of Downtown Toronto. This is to keep the resulting data relatively small and easier to manipulate for learning purposes.

In [19]:
#Create a dataframe with only the borough of Downtown Toronto
downtown = df_whole[df_whole['Borough'] == 'Downtown Toronto'].reset_index()
downtown.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Map new toronto dataframe

In [24]:
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown['Latitude'], downtown['Longitude'], downtown['Borough'], downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Exploring Using Foursquare

In [26]:
#Foursquare credentials
CLIENT_ID = '2XAVNQNEMFRZHF4QDDUW0V4H0FOH3MR2OBQF24SADGXDXQGY'
CLIENT_SECRET = 'XJCD4UZMTEF1O2U3L1AQERG1L3HZHUSDEIPQJCAFKSAMIKWG'
VERSION = '20180605' # Foursquare API version

Define Functions to Get Venue Data


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Get Downtown Toronto Venues

I am doing this by postal code since each postal code can have multiple neighborhoods.

In [40]:
LIMIT = 100
downtown_venues = getNearbyVenues(names = downtown['PostalCode'], 
                                  latitudes = downtown['Latitude'], 
                                  longitudes = downtown['Longitude'])

M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M5W
M5X
M6G


In [41]:
print(downtown_venues.shape)
downtown_venues.head()

(1286, 7)


,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4W,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,M4W,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,M4W,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,M4W,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,M4X,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [46]:
downtown_venues.groupby('PostalCode').count().sort_values(by='Venue', ascending = False)

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M5J,100,100,100,100,100,100
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5H,100,100,100,100,100,100
M5K,100,100,100,100,100,100
M5L,100,100,100,100,100,100
M5T,100,100,100,100,100,100
M5X,100,100,100,100,100,100
M5W,91,91,91,91,91,91


It looks like Postal Codes M5J, M5B, M5C, M5H, M5K, M5L, M5T, M5X have the most venues in downtown Toronto while M4W has the least.

In [47]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 206 uniques categories.


Analyze the Venues of each postal code.

In [53]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['PostalCode'] = downtown_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

print('Onehot Shape: {}'.format(downtown_onehot.shape))
downtown_groups = downtown_onehot.groupby('PostalCode').mean().reset_index()
print('Grouped shape: {}'.format(downtown_groups.shape))
downtown_groups.head()

Onehot Shape: (1286, 207)
Grouped shape: (18, 207)


,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4W,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.25,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
1,M4X,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.020408
2,M4Y,0.0,0.011494,0.011494,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.011494,0.011494,0.011494,0.00,0.011494,0.0,0.011494
3,M5A,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.019608
4,M5B,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.00,0.0,0.010000,0.000000,0.010000,0.01,0.000000,0.0,0.000000


In [56]:
downtown_groups.columns

Index(['PostalCode', 'Accessories Store', 'Adult Boutique',
       'Afghan Restaurant', 'Airport', 'Airport Food Court', 'Airport Gate',
       'Airport Lounge', 'Airport Service', 'Airport Terminal',
       ...
       'Toy / Game Store', 'Trail', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Wine Bar', 'Wings Joint', 'Women's Store',
       'Yoga Studio'],
      dtype='object', length=207)

Top 5 Venues in Each Postal Code

In [58]:
num_top_venues = 5

for hood in downtown_groups['PostalCode']:
    print("----"+hood+"----")
    temp = downtown_groups[downtown_groups['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
               venue  freq
0               Park  0.50
1         Playground  0.25
2              Trail  0.25
3  Accessories Store  0.00
4             Museum  0.00


----M4X----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.08
2   Indian Restaurant  0.04
3  Italian Restaurant  0.04
4              Bakery  0.04


----M4Y----
                 venue  freq
0  Japanese Restaurant  0.08
1          Coffee Shop  0.06
2              Gay Bar  0.05
3     Sushi Restaurant  0.05
4         Burger Joint  0.03


----M5A----
                venue  freq
0         Coffee Shop  0.18
1                Park  0.08
2              Bakery  0.08
3                Café  0.06
4  Mexican Restaurant  0.04


----M5B----
                 venue  freq
0          Coffee Shop  0.09
1       Clothing Store  0.07
2       Cosmetics Shop  0.04
3                 Café  0.04
4  Japanese Restaurant  0.03


----M5C----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06


Lots of food/drink places. Few parks or store show up in the top 5. Parks makes since since a postal code
is unlikely to have a high number of parks. M5V is oblivously the airport.

Turn into DataFrame

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
downtown_venues_sorted = pd.DataFrame(columns=columns)
downtown_venues_sorted['PostalCode'] = downtown_groups['PostalCode']

for ind in np.arange(downtown_groups.shape[0]):
    downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_groups.iloc[ind, :], num_top_venues)

downtown_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Coffee Shop,Restaurant,Chinese Restaurant,Pub,Bakery,Pizza Place,Café,Convenience Store,Italian Restaurant,Indian Restaurant
2,M4Y,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Men's Store,Gastropub,Café
3,M5A,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Breakfast Spot,Pub,Theater,Restaurant,Brewery
4,M5B,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Diner
5,M5C,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gastropub,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Cocktail Bar
6,M5E,Coffee Shop,Cocktail Bar,Restaurant,Café,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Bakery,Seafood Restaurant
7,M5G,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Bar,Sandwich Place,Bubble Tea Shop,Burger Joint,Indian Restaurant
8,M5H,Coffee Shop,Café,American Restaurant,Steakhouse,Gym,Thai Restaurant,Bar,Restaurant,Hotel,Deli / Bodega
9,M5J,Coffee Shop,Hotel,Café,Aquarium,Pizza Place,Sports Bar,Brewery,Scenic Lookout,Italian Restaurant,Music Venue


Downtown Toronto sure loves its coffee! Coffee shops and Cafes show up a lot. Especially in the 1st and 2nd most common venues.

### Clustering

In [65]:
# set number of clusters
kclusters = 5

downtown_groups_clustering = downtown_groups.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_groups_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 4, 2, 2, 2, 2, 2, 2], dtype=int32)

DataFrame of Downtown Clusters

In [75]:
downtown_merged = downtown

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

downtown_merged = downtown_merged.join(downtown_venues_sorted.set_index('PostalCode'), on='PostalCode')
downtown_merged.drop('index', axis = 1, inplace = True)
downtown_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,2,Coffee Shop,Restaurant,Chinese Restaurant,Pub,Bakery,Pizza Place,Café,Convenience Store,Italian Restaurant,Indian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Men's Store,Gastropub,Café
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Breakfast Spot,Pub,Theater,Restaurant,Brewery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Diner


All about that coffee and food! The M4W cluster seems to be more active with parks, playgrounds, trails, and yoga studios being the most common venues.

### Visualize Clusters

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['PostalCode'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

I wonder if the clustering is the best since cluster 2 seems to have many items, while the rest of the clusters have only 1. Changing the number of clusters (k) may change this. But for now, I want to examine the cluster 2 to see what it is comprised of.

In [83]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,2,Coffee Shop,Restaurant,Chinese Restaurant,Pub,Bakery,Pizza Place,Café,Convenience Store,Italian Restaurant,Indian Restaurant
2,Downtown Toronto,2,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Men's Store,Gastropub,Café
4,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Diner
5,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gastropub,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Cocktail Bar
6,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Restaurant,Café,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Bakery,Seafood Restaurant
7,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Bar,Sandwich Place,Bubble Tea Shop,Burger Joint,Indian Restaurant
8,Downtown Toronto,2,Coffee Shop,Café,American Restaurant,Steakhouse,Gym,Thai Restaurant,Bar,Restaurant,Hotel,Deli / Bodega
9,Downtown Toronto,2,Coffee Shop,Hotel,Café,Aquarium,Pizza Place,Sports Bar,Brewery,Scenic Lookout,Italian Restaurant,Music Venue
10,Downtown Toronto,2,Coffee Shop,Hotel,Café,American Restaurant,Italian Restaurant,Restaurant,Gastropub,Deli / Bodega,Sports Bar,Gym
11,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Steakhouse,Gym,Italian Restaurant,Deli / Bodega


# COFFEE!

I think downtown Toronto may have an obession.